In [1]:
import os
import requests
import pandas as pd
from datetime import datetime
import io
from dotenv import load_dotenv
import json

load_dotenv()

API_KEY = os.getenv("FMP_API_KEY")
if not API_KEY:
    raise ValueError("API_KEY를 찾을 수 없습니다. .env 파일을 확인해주세요.")
BASE_URL = "https://financialmodelingprep.com/stable"
DOWNLOAD_FOLDER = "/app/data/fmp"

def fmp_down_save(url, save_path):
    """
    파일 확장자(.csv, .json)에 따라 알맞게 저장하고 읽어오는 만능 함수
    """
    # 1. 폴더 생성
    directory = os.path.dirname(save_path)
    if directory and not os.path.exists(directory):
        os.makedirs(directory)

    # 2. 캐시 확인 (파일이 이미 있으면 로딩)
    if os.path.exists(save_path):
        print(f"✅ [Cache Hit] : {save_path}")
        try:
            if save_path.endswith('.csv'):
                return pd.read_csv(save_path)
            elif save_path.endswith('.json'):
                return pd.read_json(save_path)
            else:
                return None # 텍스트 파일 등은 DataFrame 변환 안 함
        except Exception as e:
            print(f"⚠️ 파일 읽기 실패 (손상 가능성 있음): {e}")
            # 파일이 깨졌을 경우 삭제 후 다시 받는 로직을 추가할 수도 있음
            return None

    # 3. 다운로드
    print(f"⬇️ [Download] : {url}")
    try:
        # timeout을 설정하여 무한 대기 방지 (30초)
        response = requests.get(url, timeout=30)
        response.raise_for_status() # 404, 500 등 에러 발생 시 즉시 예외 처리
        
        # 4. 저장
        with open(save_path, 'wb') as f:
            f.write(response.content)
        print(f"💾 [Saved] : {save_path}")
        
        # 5. 읽어서 반환
        if save_path.endswith('.csv'):
            # API 데이터가 UTF-8이라고 가정
            return pd.read_csv(io.BytesIO(response.content), encoding='utf-8')
        elif save_path.endswith('.json'):
            return pd.read_json(io.BytesIO(response.content))
        
    except requests.exceptions.Timeout:
        print(f"❌ [Error] 시간 초과 (Timeout): {url}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"❌ [Error] 네트워크/요청 오류: {e}")
        return None
    except Exception as e:
        print(f"❌ [Error] 알 수 없는 오류: {e}")
        return None

### 1. 재무제표 데이터 ###
#(사용불가 / Ultimate Plan 구독 필요)A. 특정 연도의 모든 기업 재무제표 다운로드
def finanacial_data_year_income(year, period="quarter"):
    """특정 연도의 모든 기업의 손익계산서 다운로드"""
    try: 
        url = f"{BASE_URL}/income-statement-bulk?year={year}&period={period}&apikey={API_KEY}"
        download_path = f"financial/year/{year}_income.csv"
        final_path = os.path.join(DOWNLOAD_FOLDER, download_path)
        fmp_down_save(url, final_path)
    except Exception as e:
        print(f"Error in income: {e}")

def finanacial_data_year_balance_sheet(year, period="quarter"):
    """특정 연도의 모든 기업의 대차대조표 다운로드"""
    try: 
        url = f"{BASE_URL}/balance-sheet-statement-bulk?year={year}&period={period}&apikey={API_KEY}"
        download_path = f"financial/year/{year}_balance_sheet.csv"
        final_path = os.path.join(DOWNLOAD_FOLDER, download_path)
        fmp_down_save(url, final_path)
    except Exception as e:
        print(f"Error in balance: {e}")

def finanacial_data_year_cash_flow(year, period="quarter"):
    """특정 연도의 모든 기업의 현금흐름표 다운로드"""
    try: 
        url = f"{BASE_URL}/cash-flow-statement-bulk?year={year}&period={period}&apikey={API_KEY}"
        download_path = f"financial/year/{year}_cash_flow.csv"
        final_path = os.path.join(DOWNLOAD_FOLDER, download_path)
        fmp_down_save(url, final_path)
    except Exception as e:
        print(f"Error in cash_flow: {e}")

#B. 특정 종목의 과거 전체 데이터 다운로드
def finanacial_data_ticker_income(ticker, period="quarter", limit=1000):
    """특정 종목의 과거 전체 손익계산서 다운로드"""
    try: 
        url = f"{BASE_URL}/income-statement/{ticker}?period={period}&limit={limit}&apikey={API_KEY}"
        download_path = f"financial/ticker/{ticker}_income.json"
        final_path = os.path.join(DOWNLOAD_FOLDER, download_path)
        fmp_down_save(url, final_path)
    except Exception as e:
        print(e)

def finanacial_data_ticker_balance_statement(ticker, period="quarter", limit=1000):
    """특정 종목의 과거 전체 대차대조표 다운로드"""
    try: 
        url = f"{BASE_URL}/balance-sheet-statement/{ticker}?period={period}&limit={limit}&apikey={API_KEY}"
        download_path = f"financial/ticker/{ticker}_balance_sheet.json"
        final_path = os.path.join(DOWNLOAD_FOLDER, download_path)
        fmp_down_save(url, final_path)
    except Exception as e:
        print(e)

def finanacial_data_ticker_cash_flow(ticker, period="quarter", limit=1000):
    """특정 종목의 과거 전체 현금흐름표 다운로드"""
    try: 
        url = f"{BASE_URL}/cash-flow-statement/{ticker}?period={period}&limit={limit}&apikey={API_KEY}"
        download_path = f"financial/ticker/{ticker}_cash_flow.json"
        final_path = os.path.join(DOWNLOAD_FOLDER, download_path)
        fmp_down_save(url, final_path)
    except Exception as e:
        print(e)

### 2. 가격 데이터 ###
#A.특정 종목의 전 기간 데이터
def price_data_ticker(ticker, start_date, end_date):
    """특정 종목의 전 기간 데이터"""
    try: 
        url = f"{BASE_URL}/historical-price-full/{ticker}?from={start_date}&to={end_date}&apikey={API_KEY}"
        download_path = f"price/ticker/{ticker}_{start_date}_to_{end_date}.json"
        final_path = os.path.join(DOWNLOAD_FOLDER, download_path)
        fmp_down_save(url, final_path)
    except Exception as e:
        print(e)

#B.특정 날짜의 전 종목 데이터
def price_data_date(date):
    """특정 날짜의 전 종목 데이터 (입력 date는 문자열 'YYYY-MM-DD' 또는 datetime 객체)"""
    try: 
        # date가 datetime 객체라면 문자열로 변환, 문자열이면 그대로 사용
        if isinstance(date, datetime):
            date_str = date.strftime("%Y-%m-%d")
        else:
            date_str = str(date)
            
        url = f"{BASE_URL}/batch-request-eod-prices?date={date_str}&apikey={API_KEY}"
        
        # 날짜 문자열에서 연도 추출 (YYYY-MM-DD 형식 가정)
        year = date_str.split('-')[0]
        
        download_path = f"price/date/{year}/{date_str}.json"
        final_path = os.path.join(DOWNLOAD_FOLDER, download_path)
        fmp_down_save(url, final_path)
    except Exception as e:
        print(f"Error in price_data_date: {e}")

### 3. 시가총액 정보 ###
#A.(사용불가 / Ultimate Plan 구독 필요)특정 연도의 모든 기업 시가총액 & 투자지표(PER, PBR 등)
def metrics_data_year(year, period = "quarter"):
    """특정 연도의 모든 기업 시가총액 & 투자지표(PER, PBR 등)"""
    try: 
        url = f"{BASE_URL}/key-metrics-bulk?year={year}&period={period}&apikey={API_KEY}"
        download_path = f"metrics/year/{year}.csv"
        final_path = os.path.join(DOWNLOAD_FOLDER, download_path)
        return fmp_down_save(url, final_path)
    except Exception as e:
        print(e)

#B. 특정 종목의 일별 시가총액 전 기간 데이터 다운
def market_cap_data_ticker_date(ticker, start_date, end_date, limit=100):
    """
    특정 종목의 일별 시가총액 역사적 데이터를 다운로드합니다.
    limit=5000 설정 시 주말 제외 약 20년치 데이터를 가져옵니다.
    """
    try:
        url = f"{BASE_URL}/historical-market-capitalization?symbol={ticker}&from={start_date}&to={end_date}&limit={limit}&apikey={API_KEY}"
        download_path = f"market_cap/{ticker}_{start_date}_to_{end_date}.json"
        final_path = os.path.join(DOWNLOAD_FOLDER, download_path)
        return fmp_down_save(url, final_path)
    except Exception as e:
        print(f"❌ {ticker} Daily Market Cap 다운로드 실패: {e}")

### 4. 상장폐지된/거래중인 종목 리스트 ###
#A.현재 거래중인 종목 리스트
def stock_list_data():
    """현재 거래중인 종목 리스트"""
    try: 
        url = f"{BASE_URL}/stock-list?apikey={API_KEY}"
        download_path = f"stock-list/stock-list.json"
        final_path = os.path.join(DOWNLOAD_FOLDER, download_path)
        return fmp_down_save(url, final_path)
    except Exception as e:
        print(e)

#B.상장폐지된 종목 리스트
def delisted_companies_data(page = 0, limit = 5000):
    """상장 폐지된 종목 리스트"""
    try: 
        url = f"{BASE_URL}/delisted-companies?page={page}&limit={limit}&apikey={API_KEY}"
        download_path = f"delisted_companies/{page}.json"
        final_path = os.path.join(DOWNLOAD_FOLDER, download_path)
        return fmp_down_save(url, final_path)
    except Exception as e:
        print(e)

#C.Screener 활용하여 특정 거래소 거래중인 종목 리스트 불러오기
def stock_screener_exchanges_stock_list(exchange, limit = 18800):
    """Stock Screener활용하여 특정 거래소의 거래중인 종목 리스트"""
    try: 
        url = f"{BASE_URL}/company-screener?exchange={exchange}&limit={limit}&apikey={API_KEY}"
        download_path = f"stock-list/{exchange}.json"
        final_path = os.path.join(DOWNLOAD_FOLDER, download_path)
        return fmp_down_save(url, final_path)
    except Exception as e:
        print(e)

### 5. 특정 회사의 상세 정보 불러오기 ###
def company_profile_data_ticker(ticker):
    """특정 회사의 상세 프로파일 정보 불러오기"""
    try: 
        url = f"{BASE_URL}/profile?symbol={ticker}&apikey={API_KEY}"
        download_path = f"company_profile/{ticker}.json"
        final_path = os.path.join(DOWNLOAD_FOLDER, download_path)
        return fmp_down_save(url, final_path)
    except Exception as e:
        print(e)

In [2]:
#특정 날짜의 주가 데이터 불러오기
    #
#목표하는 연도의 모든 기업의 시가총액 불러오기
    #원하는 거래소만 필터링
#시가총액 기준 50% 필터링하여 티커 리스트 생성
#티커 리스트에 있는 기업들의 재무제표 전체 불러오기 (중복 호출 방지)
    #각 종목의 재무제표 NULL값 / date값 조정

In [3]:
# 타겟 날짜 설정 / 티커 리스트 불러오기
# 각 종목의 과거 시가총액 / 재무제표 / 가격 데이터 다운로드

In [4]:
# 리밸런싱 날짜 설정
# 각 팩터 계산하기

In [5]:
#과거~현재까지 거래중인 주식 PIT 캘린더 생성하는 함수

def create_universe_calendar(exchanges):
    # screener 활용하여 현재 시점에 거래중인 거래소 stock list 다운로드
    all_stock_list = []
    for exchange in exchanges:
        stock_list = stock_screener_exchanges_stock_list(exchange)
        all_stock_list.append(stock_list)
    
    #정리 및 필터링
    final_df = pd.concat(all_stock_list,ignore_index=True)
    #ETF / Fund 제외
    final_df = final_df[(final_df['isEtf'] == False) & (final_df['isFund'] == False)]
    #중복 제거
    final_df = final_df.drop_duplicates(subset=['symbol'], keep='first')
    final_df = final_df.reset_index(drop=True)

    alive_ticker = final_df['symbol'].copy()

    # 현재 거래중인 주식들의 profile 정보 모으기
    all_df_list = []
    for ticker in alive_ticker:
        profile_df = company_profile_data_ticker(ticker)
        if profile_df is not None and not profile_df.empty:
            all_df_list.append(profile_df)

    if all_df_list:
        all_profile_df = pd.concat(all_df_list,ignore_index=True)
    else:
        all_profile_df = pd.DataFrame()
    
    #profile 정보로부터 필요한 정보만 추출
    alive_ipo_df = all_profile_df[['symbol', 'ipoDate', 'exchange']].copy()
    alive_ipo_df['delisted_date'] = None
    alive_ipo_df = alive_ipo_df.rename(columns={'ipoDate':'ipo_date'})
    alive_ipo_df = alive_ipo_df[['symbol','ipo_date','delisted_date','exchange']]

    #모든 상장 폐지 종목들의 정보 가져오기
    all_delisted_data = []
    pages = list(range(0,100))
    for page in pages:
        df = delisted_companies_data(page)
        if not df.empty:
            all_delisted_data.append(df)
    delisted_ipo_df = pd.concat(all_delisted_data, ignore_index=True)

    #필요한 정보만 추출
    delisted_ipo_df = delisted_ipo_df[['symbol','ipoDate','delistedDate','exchange']].copy()
    delisted_ipo_df = delisted_ipo_df.rename(columns={'ipoDate':'ipo_date', 'delistedDate':'delisted_date'})
    #지정한 거래소만 필터링
    delisted_ipo_df = delisted_ipo_df[delisted_ipo_df['exchange'].isin(exchanges)]

    #모두 합치기 
    all_ipo_delisted_df = pd.concat([alive_ipo_df,delisted_ipo_df],ignore_index=True)

    # 날짜 형식 변환 
    all_ipo_delisted_df['ipo_date'] = pd.to_datetime(all_ipo_delisted_df['ipo_date'])
    all_ipo_delisted_df['delisted_date'] = pd.to_datetime(all_ipo_delisted_df['delisted_date'])
    
    #결측치 및 전처리
    today = pd.Timestamp.now().normalize()
    all_ipo_delisted_df['delisted_date'] = all_ipo_delisted_df['delisted_date'].fillna(today)
    all_ipo_delisted_df = all_ipo_delisted_df.dropna(subset=['ipo_date'])

    #테스트 
    fl_check = all_ipo_delisted_df[all_ipo_delisted_df['symbol'] == 'FL']
    print(fl_check)

    #전체 기간 설정
    min_date = all_ipo_delisted_df['ipo_date'].min()
    max_date = pd.Timestamp.now().normalize()
    
    #전체 날짜 생성
    total_days = pd.date_range(start=min_date, end=max_date)
    total_days_str = total_days.strftime('%Y-%m-%d')

    #빈 캘린더 생성
    #pit_calendar = {d_str : [] for d_str in total_days_str}
    pit_calendar = {d_str : set() for d_str in total_days_str}

    #종목별 루프
    for row in all_ipo_delisted_df.itertuples():
        ticker = row.symbol
        start_date = row.ipo_date
        end_date = row.delisted_date
        
        #날짜 범위 생성
        active_date = pd.date_range(start=start_date,end=end_date)
        active_date_str = active_date.strftime('%Y-%m-%d')

        for d_str in active_date_str:
            if d_str in pit_calendar:
                #pit_calendar[d_str].append(ticker) 
                pit_calendar[d_str].add(ticker)
                
    #집합에서 리스트로 변환
    final_calendar = {k: list(v) for k, v in pit_calendar.items()}
    
    #저장 경로 설정
    date_str = max_date.strftime('%Y-%m-%d')
    exchanges_str = "_".join(exchanges)
    calendar_path = os.path.join(DOWNLOAD_FOLDER, f"universe_calendar/{date_str}_{exchanges_str}.json")
    
    # 폴더가 없으면 생성
    os.makedirs(os.path.dirname(calendar_path), exist_ok=True)
    
    try:
        with open(calendar_path, 'w', encoding='utf-8') as f:
            # indent=4를 빼면 용량은 줄지만 사람이 읽기 힘듦. 확인용으로 넣는 게 좋음.
            json.dump(final_calendar, f, ensure_ascii=False, indent=4) 
        print(f"✅ 캘린더 저장 완료: {calendar_path}")
        
    except Exception as e:
        print(f"❌ 캘린더 저장 실패: {e}")
    
    return final_calendar

In [6]:
exchanges = ["NASDAQ", "NYSE", "AMEX"]
calendar = create_universe_calendar(exchanges)

✅ [Cache Hit] : /app/data/fmp/stock-list/NASDAQ.json
✅ [Cache Hit] : /app/data/fmp/stock-list/NYSE.json
✅ [Cache Hit] : /app/data/fmp/stock-list/AMEX.json
✅ [Cache Hit] : /app/data/fmp/company_profile/NVDA.json
✅ [Cache Hit] : /app/data/fmp/company_profile/AAPL.json
✅ [Cache Hit] : /app/data/fmp/company_profile/GOOG.json
✅ [Cache Hit] : /app/data/fmp/company_profile/GOOGL.json
✅ [Cache Hit] : /app/data/fmp/company_profile/MSFT.json
✅ [Cache Hit] : /app/data/fmp/company_profile/AMZN.json
✅ [Cache Hit] : /app/data/fmp/company_profile/META.json
✅ [Cache Hit] : /app/data/fmp/company_profile/AVGO.json
✅ [Cache Hit] : /app/data/fmp/company_profile/TSLA.json
✅ [Cache Hit] : /app/data/fmp/company_profile/WMT.json
✅ [Cache Hit] : /app/data/fmp/company_profile/ASML.json
✅ [Cache Hit] : /app/data/fmp/company_profile/MU.json
✅ [Cache Hit] : /app/data/fmp/company_profile/COST.json
✅ [Cache Hit] : /app/data/fmp/company_profile/AMD.json
✅ [Cache Hit] : /app/data/fmp/company_profile/NFLX.json
✅ [Cache

In [7]:
total_len = sum(len(v) for v in calendar.values())
print(total_len)

51888776


In [8]:
print(len(calendar['1916-01-19']))
print(calendar['1916-01-19'])

2
['IBM', 'FL']


In [ ]:
import json 

file_path = '/app/data/fmp/universe_calendar/2026-02-17_NASDAQ_NYSE_AMEX.json'
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

total_length = sum(len(v) for v in data.values())
print(total_length)

In [ ]:
# 1. 중복 티커 검사 수행
duplicates_found = {}

for date_str, tickers in calendar.items():
    seen = set()
    dups = []
    for t in tickers:
        if t in seen:
            dups.append(t)
        seen.add(t)
    
    if dups:
        # 중복된 티커들만 유니크하게 저장
        duplicates_found[date_str] = list(set(dups))

# 2. 결과 출력 및 리포팅
print("=" * 50)
if duplicates_found:
    print(f"⚠️ 중복 발견된 날짜 수: {len(duplicates_found)}일")
    
    # 샘플 5개만 보기 좋게 출력
    sample_dates = list(duplicates_found.keys())[:5]
    for date_str in sample_dates:
        dups = duplicates_found[date_str]
        print(f"\n📅 {date_str}: 중복 {len(dups)}건")
        print(f"   ㄴ 중복된 티커: {', '.join(dups)}")
else:
    print("✅ 데이터가 깨끗합니다: 모든 날짜에 중복 티커가 없습니다.")
print("=" * 50)